<a href="https://colab.research.google.com/github/kyrajeep/DL_Projects/blob/master/detect_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install timm
!pip install kaggle


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [3]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
#import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report


In [4]:
import os
#from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
#from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor



In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kjpark5","key":"fa8f9c9876a43e6575d18c5a34903817"}'}

In [6]:
# use the Kaggle API to directly download data
!mkdir ~/.kaggle
#files.upload()
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets list


In [ ]:
!kaggle datasets download -d andrewmvd/face-mask-detection
!unzip face-mask-detection.zip


In [8]:
!ls

annotations  face-mask-detection.zip  images  sample_data


In [9]:
images = list(sorted(os.listdir("images/")))
labels = list(sorted(os.listdir("annotations/")))
print(labels)

['maksssksksss0.xml', 'maksssksksss1.xml', 'maksssksksss10.xml', 'maksssksksss100.xml', 'maksssksksss101.xml', 'maksssksksss102.xml', 'maksssksksss103.xml', 'maksssksksss104.xml', 'maksssksksss105.xml', 'maksssksksss106.xml', 'maksssksksss107.xml', 'maksssksksss108.xml', 'maksssksksss109.xml', 'maksssksksss11.xml', 'maksssksksss110.xml', 'maksssksksss111.xml', 'maksssksksss112.xml', 'maksssksksss113.xml', 'maksssksksss114.xml', 'maksssksksss115.xml', 'maksssksksss116.xml', 'maksssksksss117.xml', 'maksssksksss118.xml', 'maksssksksss119.xml', 'maksssksksss12.xml', 'maksssksksss120.xml', 'maksssksksss121.xml', 'maksssksksss122.xml', 'maksssksksss123.xml', 'maksssksksss124.xml', 'maksssksksss125.xml', 'maksssksksss126.xml', 'maksssksksss127.xml', 'maksssksksss128.xml', 'maksssksksss129.xml', 'maksssksksss13.xml', 'maksssksksss130.xml', 'maksssksksss131.xml', 'maksssksksss132.xml', 'maksssksksss133.xml', 'maksssksksss134.xml', 'maksssksksss135.xml', 'maksssksksss136.xml', 'maksssksksss137.x

In [10]:
def generate_box(obj):

    xmin = int(obj.find('xmin').text)
    ymin = int(obj.find('ymin').text)
    xmax = int(obj.find('xmax').text)
    ymax = int(obj.find('ymax').text)

    return [xmin, ymin, xmax, ymax]

def generate_label(obj):
   # TODO: generate labels so that it matches the defined classes in DETR
    class_mapping = {
        "with_mask": 0,
        "without_mask": 1,
        "mask_worn_incorrectly": 2,
        # Add mappings for new classes
        "bag": 3,
        "glasses": 4,
        "people": 5
        }
    class_name = obj.find('name').text

    if class_name in class_mapping:
        return class_mapping[class_name]
    else:
        # If class not found in mapping, return a default label (e.g., 0)
        return 0

def generate_target(image_id, file):
    with open(file) as f:
        data = f.read()
        soup = BeautifulSoup(data, 'xml')
        objects = soup.find_all('object')

        num_objs = len(objects)
        print("number of objects found:", num_objs)
        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        labels = []
        for i in objects:
            boxes.append(generate_box(i))
            labels.append(generate_label(i))
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        # Tensorise img_id
        img_id = torch.tensor([image_id])
        # Annotation is in dictionary format
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = img_id

        return target

In [11]:
class MaskDataset(object):
    def __init__(self, transforms):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir("images/")))
#         self.labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))

    def __getitem__(self, idx):
        # load images ad masks
        file_image = 'maksssksksss'+ str(idx) + '.png'
        file_label = 'maksssksksss'+ str(idx) + '.xml'
        img_path = os.path.join("images/", file_image)
        label_path = os.path.join("annotations/", file_label)
        img = Image.open(img_path).convert("RGB")
        #Generate Label
        target = generate_target(idx, label_path)

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.imgs)



In [12]:
data_transform = transforms.Compose([
        transforms.ToTensor(),
    ])


In [13]:
#batch data with different sizes without manual padding
#https://stackoverflow.com/questions/65279115/how-to-use-collate-fn-with-dataloaders
def collate_fn(batch):
    return tuple(zip(*batch))

dataset = MaskDataset(data_transform)

In [14]:
train_size=int(len(dataset)*0.7)
test_size=len(dataset)-train_size
print('Length of dataset is', len(dataset), '\nLength of training set is :',train_size,'\nLength of test set is :', test_size)


Length of dataset is 853 
Length of training set is : 597 
Length of test set is : 256


In [15]:
trainset, testset=torch.utils.data.random_split(dataset,[train_size,test_size])

data_loader = torch.utils.data.DataLoader(
 trainset, batch_size=4, collate_fn=collate_fn)


In [16]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    img_size = imgs[0].size()
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)
    break


number of objects found: 6
number of objects found: 3
number of objects found: 1
number of objects found: 1
[{'boxes': tensor([[ 37.,  36.,  55.,  54.],
        [162.,  24., 192.,  54.],
        [176., 109., 202., 127.],
        [261.,  36., 287.,  63.],
        [312.,  33., 329.,  45.],
        [357.,  27., 371.,  42.]], device='cuda:0'), 'labels': tensor([0, 0, 0, 0, 0, 0], device='cuda:0'), 'image_id': tensor([637], device='cuda:0')}, {'boxes': tensor([[133.,  82., 157., 109.],
        [217.,  76., 243., 103.],
        [322.,  62., 341.,  83.]], device='cuda:0'), 'labels': tensor([0, 0, 0], device='cuda:0'), 'image_id': tensor([519], device='cuda:0')}, {'boxes': tensor([[ 88.,  70., 206., 195.]], device='cuda:0'), 'labels': tensor([0], device='cuda:0'), 'image_id': tensor([654], device='cuda:0')}, {'boxes': tensor([[111., 166., 199., 280.]], device='cuda:0'), 'labels': tensor([0], device='cuda:0'), 'image_id': tensor([743], device='cuda:0')}]


In [17]:

from transformers import DetrImageProcessor, DetrForObjectDetection, DetrConfig
import torch
from PIL import Image
import requests

#url = "http://images.cocodataset.org/val2017/000000039769.jpg"
#image = Image.open(requests.get(url, stream=True).raw)

# use Detr model from huggingface
# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor()
config = DetrConfig()
model = DetrForObjectDetection(config)

#inputs = processor(images=imgs, return_tensors="pt", do_rescale=False)
#outputs = model(**inputs)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [18]:
import torch.nn as nn
'''
from transformers import AutoModel, AutoConfig
# Modify the model architecture to enable finetuning

# Load pre-trained DeTR model
model_name = "facebook/detr-resnet-50"

config = AutoConfig.from_pretrained(model_name)
print(config)
model = AutoModel.from_pretrained(model_name, config=config)
'''


# Replace classification head
num_classes = 6
model.class_embed = nn.Linear(model.config.d_model, num_classes)
model.num_classes = num_classes  # Update number of classes in the config

# Freeze all layers except the final layers
for param in model.parameters():
    param.requires_grad = False

# Make final layers trainable
for param in model.class_embed.parameters():
    param.requires_grad = True

# Optionally, you may also make other final layers trainable, if applicable
# For example, if you have additional layers after class_embed, you can make them trainable:
# for param in model.additional_final_layers.parameters():
#     param.requires_grad = True

# You can verify which layers are trainable
for name, param in model.named_parameters():
    print(f"Layer: {name}, Trainable: {param.requires_grad}")

Layer: model.backbone.conv_encoder.model.conv1.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer1.0.conv1.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer1.0.conv2.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer1.0.conv3.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer1.0.downsample.0.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer1.1.conv1.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer1.1.conv2.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer1.1.conv3.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer1.2.conv1.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer1.2.conv2.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer1.2.conv3.weight, Trainable: False
Layer: model.backbone.conv_encoder.model.layer2.0.conv1.weight, Trainable: False
Layer: model.backbone.conv_enc

In [19]:
import torch.nn.functional as F

def calculate_iou(predicted_box, gt_box, pred_logits, target_logits):
    """
    Calculate IoU (Intersection over Union) between two bounding boxes.

    Arguments:
    box1: List or tuple containing [xmin, ymin, xmax, ymax] of the first box.
    box2: List or tuple containing [xmin, ymin, xmax, ymax] of the second box.

    Returns:
    IoU: Intersection over Union between the two boxes + classification loss
    """
    classification_loss = F.cross_entropy(pred_logits, target_logits)

    # Calculate intersection (area of overlap)
    x1 = max(predicted_box[0], gt_box[0])
    y1 = max(predicted_box[1], gt_box[1])
    x2 = min(predicted_box[2], gt_box[2])
    y2 = min(predicted_box[3], gt_box[3])

    intersection = max(0, abs(x2 - x1)) * max(0, abs(y2 - y1))

    # Calculate union (area of combined bounding boxes)
    area_box1 = (predicted_box[2] - predicted_box[0]) * (predicted_box[3] -predicted_box[1])
    area_box2 = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
    union = area_box1 + area_box2 - intersection

    # Calculate IoU
    iou = intersection / union if union > 0 else 0

    return iou + classification_loss

def class_loss(pred_logits, target):
    classification_loss = F.cross_entropy(pred_logits, target)


In [20]:
# Creating a data loader for the train set
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=4, collate_fn=collate_fn
)


In [23]:
from torch.optim import Adam
import torch.nn.functional as F
from tqdm import tqdm

batch_size = 4
# Define optimizer and learning rate scheduler
optimizer = Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=0.001, gamma=0.001)
num_epochs = 10
# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0.0
    #iterate by batch to produce output logits, postprocess into bounding boxes
    #and labels.
    for images, annotations in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        optimizer.zero_grad()
        images = [img.to(device) for img in images]
        inputs = processor(images=images, return_tensors="pt", do_rescale=False)
        outputs = model(**inputs)
        print(outputs)
        #target_sizes = [img.shape[-2:] for img in images]
        results = processor.post_process_object_detection(outputs)
        print(results)
        print(annotations)

        for i in range(batch_size):
            pred_boxes = results[i]["boxes"]
            pred_labels = results[i]["labels"]
            gt_boxes = torch.as_tensor(annotations[i]['boxes'])
            gt_labels = torch.as_tensor(annotations[i]['labels'])
            loss = calculate_iou(pred_boxes, gt_boxes, pred_labels, gt_labels)
            total_loss += loss

        # Backpropagation
        loss.backward()
        optimizer.step()

    # Update learning rate
    scheduler.step()

    # Compute average loss for the epoch
    avg_loss = total_loss / len(train_loader)

    # Print average loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Avg. Loss: {avg_loss:.4f}")

# Save the trained model
model.save_pretrained()


Epoch 1/10:   0%|          | 0/150 [00:00<?, ?it/s]

number of objects found: 6
number of objects found: 3
number of objects found: 1
number of objects found: 1


DetrObjectDetectionOutput(loss=None, loss_dict=None, logits=tensor([[[-0.1336, -0.0535,  0.4068],
         [-0.1517, -0.3719,  0.2063],
         [-0.3459, -0.5199,  0.1730],
         ...,
         [-0.3656, -0.4860,  0.1479],
         [-0.1874, -0.3861, -0.1354],
         [-0.3350, -0.3194,  0.3163]],

        [[-0.4425, -0.1370,  0.2409],
         [-0.2591, -0.3982,  0.1279],
         [-0.3450, -0.3206,  0.0490],
         ...,
         [-0.2772, -0.2356,  0.1142],
         [-0.1587, -0.3406, -0.0331],
         [-0.3143, -0.1647,  0.2307]],

        [[-0.4722, -0.2911,  0.3071],
         [-0.4122, -0.4554,  0.1748],
         [-0.2616, -0.4909, -0.0089],
         ...,
         [-0.2484, -0.3202,  0.1477],
         [-0.1998, -0.0858,  0.3936],
         [-0.2830, -0.2156,  0.3512]],

        [[-0.3792, -0.3246,  0.1406],
         [-0.2887, -0.2413,  0.2668],
         [-0.2287, -0.3376,  0.2715],
         ...,
         [-0.1243, -0.2976,  0.1473],
         [-0.1822, -0.2182,  0.3122],
    

RuntimeError: size mismatch (got input: [0], target: [6])

In [ ]:


# Device configuration
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
threshold = 0.5
# Ground truth and predicted labels
gt_labels_all = []
pred_labels_all = []
model.eval()

# Evaluation loop
for imgs, annotations in train_loader:
    imgs = [img.to(device) for img in imgs]
    img_size = imgs[0].size()
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    # Processing the images through the model
    inputs = processor(images=imgs, return_tensors="pt", do_rescale=False)
    outputs = model(**inputs)



    # Post-processing object detection results
    target_sizes = [img.shape[-2:] for img in imgs]
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

    # Model predictions
    pred_boxes = results["boxes"].tolist()
    pred_labels = results["labels"].tolist()

    # Ground truth annotations
    gt_boxes = [ann["boxes"].tolist() for ann in annotations]
    gt_labels = [ann["labels"].tolist() for ann in annotations]

    # Compare predictions with ground truth
    for pred_box, pred_label, gt_box, gt_label in zip(pred_boxes, pred_labels, gt_boxes, gt_labels):
        # Compare each predicted box with ground truth boxes for the same image
        for i, gt_box_i in enumerate(gt_box):
            iou = calculate_iou(pred_box, gt_box_i)  # Calculate Intersection over Union
            if iou > threshold:  # You need to define a threshold to consider a detection as correct
                pred_label_name = model.config.id2label[pred_label]
                gt_label_name = model.config.id2label[gt_label[i]]
                print(f"Prediction: {pred_box} - Label: {pred_label_name}")
                print(f"Ground Truth: {gt_box_i} - Label: {gt_label_name}")
                print(f"IoU: {iou}")
                gt_labels_all.append(gt_label_name)
                pred_labels_all.append(pred_label_name)


# with no fine-tuning and threshold at 0.6, no image passed the threshold.
# TODO: finetune

In [ ]:
# Creating a data loader for the test set
test_loader = torch.utils.data.DataLoader(
    testset, batch_size=4, collate_fn=collate_fn
)
